In [49]:
from database import database

database= database("data.db")
database.connect()
resultats=database.read_json("all_matches_of_Test6.docx.json")
resultats






,title,question,sentence,score,pts
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Développer et stimuler les compétences individ...,0.528830,29.601
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Élabore et développe des outils spécifiques à ...,0.496883,29.601
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,en démontrant une forte aptitude dans la relat...,0.478152,29.601
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Conceptualise et met en place des projets indi...,0.423361,29.601
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Le formateur/la formatrice veille à acquérir d...,0.421851,29.601
...,...,...,...,...,...
164,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Élabore des activités pour la formation pratique,0.559728,11.232
165,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Accompagne et/ou organise les activités cultur...,0.518906,11.232
166,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.336
167,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",et propose les formations continues et autres ...,0.432927,37.440


In [50]:
import spacy
nlp = spacy.load("fr_core_news_md")  # pas besoin d'exclude

def lemmatize(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

def to_lower(text):
    return text.lower()


In [51]:
resultats["sentence_lemmatized"] = resultats["sentence"].apply(to_lower)
resultats["sentence_lemmatized"] = resultats["sentence_lemmatized"].apply(lemmatize)
resultats['question_lemmatized'] = resultats['question'].apply(to_lower)
resultats['question_lemmatized'] = resultats['question_lemmatized'].apply(lemmatize)
resultats

,title,question,sentence,score,pts,sentence_lemmatized,question_lemmatized
0,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Développer et stimuler les compétences individ...,0.528830,29.601,développer et stimuler le compétence individue...,le fonction requérir le faculté de analyser un...
1,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Élabore et développe des outils spécifiques à ...,0.496883,29.601,élaborer et développer un outil spécifique à l...,le fonction requérir le faculté de analyser un...
2,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,en démontrant une forte aptitude dans la relat...,0.478152,29.601,en démontrer un fort aptitude dans le relation...,le fonction requérir le faculté de analyser un...
3,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Conceptualise et met en place des projets indi...,0.423361,29.601,conceptualis et mettre en place un projet indi...,le fonction requérir le faculté de analyser un...
4,Analyse et synthèse,La fonction requiert la faculté d'analyser des...,Le formateur/la formatrice veille à acquérir d...,0.421851,29.601,le formateur sur le formateur veille à acquéri...,le fonction requérir le faculté de analyser un...
...,...,...,...,...,...,...,...
164,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Élabore des activités pour la formation pratique,0.559728,11.232,élaborer un activité pour le formation pratique,lui avoir un rôle de animation inclure le tran...
165,Rôle dans la gestion des ressources humaines,Elle a un rôle d'animation incluant le transfe...,Accompagne et/ou organise les activités cultur...,0.518906,11.232,accompagner et / ou organiser le activité cult...,lui avoir un rôle de animation inclure le tran...
166,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.336,agir dans le respect de le charte de personnel...,lui avoir un rôle de conduite de collaborateur...
167,Rôle dans la gestion des ressources humaines,"Elle a un rôle de direction, d'organisation et...",et propose les formations continues et autres ...,0.432927,37.440,et proposer le formation continu et autre déma...,"lui avoir un rôle de direction , de organisati..."


In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [53]:
def tfidf_similarity(text1, text2):
    vect = TfidfVectorizer()
    tfidf = vect.fit_transform([text1, text2])
    return cosine_similarity(tfidf[0], tfidf[1])[0][0]

In [54]:
resultats['similarity'] = resultats.apply(lambda row: tfidf_similarity(row['sentence_lemmatized'], row['question_lemmatized']), axis=1)
resultats=resultats.sort_values(by='similarity', ascending=False)
resultats_needed = resultats[["title", "question", "sentence", "score", "pts", "similarity"]]
resultats_needed

,title,question,sentence,score,pts,similarity
105,Innovation,L'exercice de la fonction requiert la capacité...,Adapte le contenu de la formation selon les ré...,0.504594,34.7490,0.627742
109,Innovation,L'exercice de la fonction s'effectue dans le c...,afin d'ajuster le suivi de l'apprenti-e à la r...,0.424735,23.1660,0.580333
96,Impact interne des prestations,L'activité de la fonction a un impact détermin...,Adapte le contenu de la formation selon les ré...,0.286584,28.6416,0.566126
99,Impact interne des prestations,L'activité de la fonction a un impact détermin...,en appliquant les principes de la CCT-ES et le...,0.264512,28.6416,0.562420
93,Impact interne des prestations,L'activité de la fonction a un impact détermin...,afin d'ajuster le suivi de l'apprenti-e à la r...,0.312888,28.6416,0.552564
...,...,...,...,...,...,...
46,Diversité et quantité des postes à gérer,La fonction gère directement un grand nombre d...,En collaboration avec le/la responsable et ses...,0.409099,56.1600,0.074426
121,Nature des communications externes,Les communications impliquent des négociations...,en fonction des objectifs liés au projet,0.385113,110.5000,0.071866
38,Diversité des missions,L'ensemble des tâches principales qui incomben...,Accomplir sa mission,0.434104,8.5800,0.070616
112,Nature,La fonction requiert de l'expérience professio...,) et effectue une démarche personnelle de form...,0.464390,11.7000,0.059930


In [55]:
resultats_fin = resultats_needed.sort_values('similarity', ascending=False).drop_duplicates(subset=['title'], keep='first')
resultats_fin

,title,question,sentence,score,pts,similarity
105,Innovation,L'exercice de la fonction requiert la capacité...,Adapte le contenu de la formation selon les ré...,0.504594,34.74900,0.627742
96,Impact interne des prestations,L'activité de la fonction a un impact détermin...,Adapte le contenu de la formation selon les ré...,0.286584,28.64160,0.566126
83,Impact externe des prestations,La fonction a un rôle participatif dans la pre...,le/la maître/esse-formateur/trice veille à éta...,0.453116,11.11968,0.536641
13,Autonomie de décision,La fonction travaille dans le cadre de consign...,Agir dans le respect de la charte du personnel...,0.521603,46.80000,0.527322
166,Rôle dans la gestion des ressources humaines,Elle a un rôle de conduite de collaborateurs/t...,Agir dans le respect de la charte du personnel...,0.480368,24.33600,0.524596
110,Nature,La fonction requiert d'avoir eu le même type d...,Diplôme de travailleur-euse social-e de niveau...,0.564396,39.00000,0.469249
146,Responsabilités de planification et de réalisa...,La fonction est responsable de l'élaboration e...,Être garant du respect de son horaire de trava...,0.417713,61.77599,0.420118
58,Durée,Une durée minimum de plus de 8 ans d'expérienc...,le/la maître/esse-formateur/trice veille à éta...,0.367461,26.00000,0.401726
130,Nature des communications internes,Les communications impliquent des échanges d'i...,en démontrant une forte aptitude dans la relat...,0.470938,66.30000,0.396536
141,Responsabilités budgétaires,La fonction participe de manière déterminante ...,Participer à la bonne gestion administrative e...,0.680958,44.92800,0.392504


In [56]:
resultats_fin.to_json("results/after_comparaison_lexicale_of_Test6.docx.json", orient="records", force_ascii=False, indent=4)